# ЗНАКОМСТВО С PANDAS

Импортируем библиотеку pandas. Считываем датасет с разделителем `;`. Сохраяняем считанный датасет в bookings.

In [1]:
import pandas as pd
bookings = pd.read_csv ('C:/Users/PC/learning/karpov/projects_karpovcourses/files/2_bookings.csv', sep=';')

Посмотрим на считанные данные.

In [2]:
bookings.head(5)

,Hotel,Is Canceled,Lead Time,arrival full date,Arrival Date Year,Arrival Date Month,Arrival Date Week Number,Arrival Date Day of Month,Stays in Weekend nights,Stays in week nights,...,Adults,Children,Babies,Meal,Country,Reserved Room Type,Assigned room type,customer type,Reservation Status,Reservation status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03


Посмотрим на размеры таблицы.

In [3]:
rows, cols = bookings.shape
print(f'{rows = }, {cols = }')

rows = 119390, cols = 21


Посмотрим на типы данных.

In [4]:
bookings.dtypes.sort_values()

stays total nights             int64
Is Canceled                    int64
Lead Time                      int64
Arrival Date Year              int64
Arrival Date Week Number       int64
Arrival Date Day of Month      int64
Stays in Weekend nights        int64
Stays in week nights           int64
Adults                         int64
Babies                         int64
Children                     float64
customer type                 object
Assigned room type            object
Reserved Room Type            object
Country                       object
Hotel                         object
Reservation Status            object
Arrival Date Month            object
arrival full date             object
Meal                          object
Reservation status_date       object
dtype: object

Названия колонок были записаны в неудобном для дальнейшей работы формате. Исправим ситуацию, приведя названия столбцов к нижнему регистру и заменив пробелы на знак нижнего подчеркивания. 

In [5]:
def replace_space(name):
    new_name = name.replace(' ', '_').lower()
    return new_name
bookings = bookings.rename(columns=replace_space)
bookings.head(1)

,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,adults,children,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01


Посмотрим, пользователи из каких стран совершили наибольшее число успешных бронирований.

In [6]:
bookings.query('is_canceled == 0') \
    .groupby(['country'], as_index=False) \
    .aggregate({'is_canceled': 'count'}) \
    .sort_values('is_canceled', ascending=False) \
    .rename(columns={'is_canceled' : 'success_orders'}) \
    .head()

,country,success_orders
125,PRT,21071
57,GBR,9676
54,FRA,8481
50,ESP,6391
42,DEU,6069


Мы видим, что наибольшее число успешных бронирований было совершено в Португалии.

Посмотрим, на сколько ночей в среднем бронируют отели типа City Hotel и Resort Hotel

In [9]:
bookings.groupby(['hotel'], as_index=False) \
    .aggregate({'stays_total_nights': 'mean'}) \
    .rename(columns={'stays_total_nights' : 'mean_nights'}) \
    .round(2)

,hotel,mean_nights
0,City Hotel,2.98
1,Resort Hotel,4.32


В среднем Resort Hotel бронируют на более долгий срок.

Иногда тип номера, присвоенного клиенту, отличается от изначально забронированного. Такое может произойти, например, по причине овербукинга. Посчитаем, сколько подобных наблюдений встретилось в нашем датасете.

In [11]:
bookings.query('assigned_room_type != reserved_room_type').shape[0]

14917

А теперь проанализируем даты запланированного прибытия. Нас интересует, на какой месяц чаще всего оформляли бронь в 2016 году, и изменился ли самый популярный месяц в 2017 году.

In [12]:
bookings.query('arrival_date_year == 2016').arrival_date_month.value_counts().sort_values(ascending=False)

October      6203
May          5478
April        5428
September    5394
June         5292
August       5063
March        4824
July         4572
November     4454
February     3891
December     3860
January      2248
Name: arrival_date_month, dtype: int64

In [13]:
bookings.query('arrival_date_year == 2017').arrival_date_month.value_counts().sort_values(ascending=False)

May         6313
April       5661
June        5647
July        5313
March       4970
August      4925
February    4177
January     3681
Name: arrival_date_month, dtype: int64

В 2016 году самым популярным месяцом был октябь, а в 2017 году - май. 

Проверим по годам, на какой месяц бронирования отеля типа City Hotel отменялись чаще всего в 2015? 2016? 2017? 

In [17]:
bookings.query('hotel == "City Hotel" and is_canceled == 0') \
    .groupby('arrival_date_year')['arrival_date_month'] \
    .value_counts()

arrival_date_year  arrival_date_month
2015               October               2065
                   September             1986
                   August                1248
                   December               986
                   November               934
                   July                   459
2016               September             2304
                   October               2272
                   May                   2240
                   June                  2203
                   August                2131
                   July                  2088
                   April                 2022
                   March                 1938
                   November              1762
                   February              1441
                   December              1406
                   January                926
2017               May                   2339
                   July                  2235
                   June                  2

В 2015 - в октябре, 2016 - в сентябре, 2017 - в мае.

Посмотрим на числовые характеристики трёх колонок: adults, children и babies. Нам интересно, какая из них имеет наибольшее среднее значение?

In [18]:
bookings[['adults', 'children', 'babies']].mean().idxmax()

'adults'

Давайе узнаем, на отели какого типа в среднем приходится больше детей.

In [19]:
bookings['total_kids'] = bookings.children + bookings.babies

In [20]:
bookings.groupby('hotel').agg({'total_kids': 'mean'}).round(2)

,total_kids
hotel,
City Hotel,0.10
Resort Hotel,0.14


Мы видим, что курортные отели посещают больше детей. 

Не все бронирования завершились успешно, поэтому можно посчитать, сколько клиентов было потеряно в процессе. Иными словами, посчитать метрику под названием Churn Rate.

Churn rate (отток, коэффициент оттока) – это процент подписчиков (например, на push-уведомления от сайта), которые отписались от канала коммуникации, отказались от услуг сервиса в течение определенного периода времени. Иными словами, представляет собой отношение количества ушедших пользователей к общему количеству пользователей, выраженное в процентах.

В нашем случае Churn Rate - это процент клиентов, которые отменили бронирование. Давайте посмотрим, как эта метрика связана с наличием детей у клиентов!

In [22]:
bookings['has_kids'] = bookings.total_kids > 0

In [23]:
bookings['has_kids'].value_counts()

False    110058
True       9332
Name: has_kids, dtype: int64

In [24]:
round((bookings[['has_kids', 'is_canceled']].value_counts() / bookings[['has_kids']].value_counts() * 100), 2)

has_kids  is_canceled
False     0              62.78
          1              37.22
True      0              65.08
          1              34.92
dtype: float64

Для клиентов без детей бронирования отменяются в разы чаще!